In [1]:
import pandas as pd
import numpy as np

import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)\

import facts
from facts.clean import clean_dataset
from facts import valid_ifthens_with_coverage_correctness, rules2rulesbyif
from facts.models import customLogisticRegression
from facts.parameters import ParameterProxy
from facts.formatting import recourse_report_reverse, print_recourse_report, print_recourse_report_cumulative, print_recourse_report_KStest_cumulative
from facts.utils import load_rules_by_if
from aif360.sklearn.datasets import fetch_compas
from facts.fairness_metrics_aggr import get_diff_table, get_comb_df, get_analysis_df

In [2]:
cor_thres = 0.5
cost_budget = 10
top_count = 20
c_inf = 5

In [3]:
X, y = fetch_compas()
X['target'] = y.values

sensitive_attribute = "race"
df = clean_dataset(X,'compas')
df.head()

,sex,age_cat,race,juv_fel_count,juv_misd_count,juv_other_count,priors_count,c_charge_degree,target
0,Male,25 - 45,African-American,0,0,0,"(-0.1, 1.0]",F,0
1,Male,Less than 25,African-American,0,0,1,"(1.0, 5.0]",F,0
2,Male,25 - 45,Caucasian,0,0,0,"(10.0, 15.0]",F,0
3,Female,25 - 45,Caucasian,0,0,0,"(-0.1, 1.0]",M,1
4,Male,25 - 45,Caucasian,0,0,0,"(-0.1, 1.0]",F,1


In [4]:
y = df['target']
X = df.drop('target', axis=1)

num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = []

In [5]:
rules_with_atomic_correctness = load_rules_by_if("rulesCompas_atomic.data")
rules_with_cumulative_correctness = load_rules_by_if("rulesCompas_cumulative.data")

# Old Metrics

In [6]:
num_features = X._get_numeric_data().columns.to_list()
cate_features = X.select_dtypes(include=['object','category']).columns.to_list()
ord_features = []
feature_weights = {'sex':100,	'age_cat':10,	'race':100,	'juv_fel_count':1,	'juv_misd_count':1,	'juv_other_count':1,	'priors_count':1,	'c_charge_degree':1}

features_with_binary_cost = cate_features
features_with_proportional_cost = num_features


comparators = facts.feature_change_builder(
    X,
    num_cols=features_with_proportional_cost,
    cate_cols=features_with_binary_cost,
    ord_cols=ord_features,
    feature_weights=feature_weights,
    num_normalization=True,
    #feats_to_normalize = ["capital-gain","capital-loss"]
)
params = ParameterProxy(featureChanges=comparators)

## Weighted Average

In [7]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric = "weighted-average",
    sort_strategy = "abs-diff-decr",
    top_count = 50,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If juv_fel_count = 1, juv_misd_count = 0, juv_other_count = 0:
	Protected Subgroup 'African-American', 4.25% covered
		Make juv_fel_count = 0 with effectiveness 4.17%.
		Aggregate cost of the above recourses = -0.42
	Protected Subgroup 'Caucasian', 3.73% covered
		Make juv_fel_count = 0 with effectiveness 60.00%.
		Aggregate cost of the above recourses = -6.00
	Bias against African-American. Unfairness score = 5.58.
If juv_fel_count = 1, juv_misd_count = 0, juv_other_count = 0, sex = Male:
	Protected Subgroup 'African-American', 4.07% covered
		Make juv_fel_count = 0 with effectiveness 4.35%.
		Aggregate cost of the above recourses = -0.43
	Protected Subgroup 'Caucasian', 3.73% covered
		Make juv_fel_count = 0 with effectiveness 60.00%.
		Aggregate cost of the above recourses = -6.00
	Bias against African-American. Unfairness score = 5.57.
If juv_fel_count = 0, juv_misd_count = 0, juv_other_count = 1, sex = Male:
	Protected Subgroup 'African-American', 6.90% covered
		Make juv_other_co

## Minimum Cost Above Threshold -- Equal Cost of Effectiveness (Macro)

In [8]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="min-above-thr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",        
        # "remove-fair-rules",
        "keep-only-min-change"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If age_cat = 25 - 45, c_charge_degree = M, priors_count = (1.0, 5.0]:
	Protected Subgroup 'African-American', 8.14% covered
		Make priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Caucasian', 1.49% covered
		Make age_cat = Greater than 45, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Aggregate cost of the above recourses = 11.00
	Bias against Caucasian. Unfairness score = 10.
If age_cat = 25 - 45, c_charge_degree = M, priors_count = (5.0, 10.0], sex = Male:
	Protected Subgroup 'African-American', 4.42% covered
		Make priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Aggregate cost of the above recourses = 1.00
	Protected Subgroup 'Caucasian', 2.24% covered
		Make age_cat = Greater than 45, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Aggregate cost of the above recourses = 11.00
	Bias against Caucasian. Unfairness score = 10.
If age_cat = Less than 25, juv_misd_count = 0, juv_other_

## Number of Rules Above Threshold -- Equal Choice for Recourse

In [9]:
top_rules, subgroup_costs = facts.select_rules_subset(
    rules_with_atomic_correctness,
    metric="num-above-thr",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=50,
    cor_threshold=0.7,
    filter_sequence = [
        "remove-contained",
        "remove-below-thr",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True
)

If age_cat = 25 - 45, c_charge_degree = M, priors_count = (1.0, 5.0]:
	Protected Subgroup 'African-American', 8.14% covered
		Make c_charge_degree = F, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Make priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Make age_cat = Greater than 45, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Make age_cat = Greater than 45, c_charge_degree = F, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Make age_cat = Greater than 45, c_charge_degree = F with effectiveness 100.00%.
		Make age_cat = Less than 25, priors_count = (-0.1, 1.0] with effectiveness 93.48%.
		Make age_cat = Greater than 45 with effectiveness 100.00%.
		Aggregate cost of the above recourses = -7.00
	Protected Subgroup 'Caucasian', 1.49% covered
		Make age_cat = Greater than 45, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Make age_cat = Greater than 45, c_charge_degree = F, priors_count = (-0.1, 1.0] with effectiveness 100.00%.
		Aggregat

In [10]:
# rules_with_cumulative = facts.cum_corr_costs_all(rules_by_if, X_test, model, sensitive_attribute="race", params=params)
# from facts.utils import load_rules_by_if, save_rules_by_if
# save_rules_by_if("rulesCompas_atomic.data", rules_by_if)
# save_rules_by_if("rulesCompas_cumulative.data", rules_with_cumulative)

# New Metrics

## Fairness of Mean Recourse Cost ($c_\infty$ implementation) -- Equal Mean Recourse

In [11]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-cinf",
    c_inf=c_inf,
    sort_strategy="generic-sorting",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If age_cat = 25 - 45, c_charge_degree = F, juv_fel_count = 0, juv_other_count = 1, sex = Male:
	Protected Subgroup 'African-American', 3.54% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 0.14.
		Make c_charge_degree = M, juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 1.14.
		Make age_cat = Less than 25, juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 25.00% and counterfactual cost = 10.14.
		Make age_cat = Less than 25, c_charge_degree = M, juv_other_count = 0 with effectiveness 25.00% and counterfactual cost = 11.14.
		Make age_cat = Greater than 45, c_charge_degree = M, juv_other_count = 0 with effectiveness 25.00% and counterfactual cost = 11.14.
		Aggregate cost of the above recourses = 44.32
	Protected Subgroup 'Caucasian', 1.49% covered
		Make juv_other_count = 0 with effectiveness 100.00% and counterfactual cost = 

## Fairness of Mean Recourse Cost (conditional expectation implementation) -- Equal Conditional Mean Recourse

In [12]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="fairness-of-mean-recourse-conditional",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If age_cat = 25 - 45, juv_other_count = 1, sex = Male:
	Protected Subgroup 'African-American', 5.66% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 0.14.
		Make age_cat = Less than 25, juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 21.88% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 10.14
	Protected Subgroup 'Caucasian', 2.99% covered
		Make juv_other_count = 0 with effectiveness 75.00% and counterfactual cost = 0.14.
		Make age_cat = Less than 25, juv_other_count = 0 with effectiveness 75.00% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 75.00% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 0.14
	Bias against African-American. Unfairness score = 10.0.
If age_cat = 25 - 45, juv_fel_count = 0, juv_other_count = 1, sex = Male:
	Pro

## Fairness of Recourse at Effectiveness Level -- Equal Cost of Effectiveness (Micro)

In [13]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="min-above-corr",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cor_threshold = cor_thres,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-cheap-rules-above-thr-cor",
        # "keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True
)

If age_cat = Less than 25, juv_other_count = 1:
	Protected Subgroup 'African-American', 4.60% covered
		Make juv_other_count = 0 with effectiveness 11.54% and counterfactual cost = 0.14.
		Make age_cat = 25 - 45, juv_other_count = 0 with effectiveness 42.31% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 84.62% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 10.14
	Protected Subgroup 'Caucasian', 13.43% covered
		Make juv_other_count = 0 with effectiveness 61.11% and counterfactual cost = 0.14.
		Make age_cat = 25 - 45, juv_other_count = 0 with effectiveness 94.44% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 100.00% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 0.14
	Bias against African-American. Unfairness score = 10.0.
If age_cat = Less than 25, juv_other_count = 1, sex = Male:
	Protected Subgroup 'African-Am

## Fairness of Effectiveness -- Equal Effectiveness

In [14]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="total-correctness",
    sort_strategy="generic-sorting-ignore-forall-subgroups-empty",
    top_count=top_count,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "keep-only-min-change"
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric=True
)

If c_charge_degree = F, juv_fel_count = 0, juv_misd_count = 0, priors_count = (-0.1, 1.0], sex = Male:
	Protected Subgroup 'African-American', 15.93% covered
		Make priors_count = (1.0, 5.0] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make c_charge_degree = M with effectiveness 90.00% and counterfactual cost = 1.0.
		Make c_charge_degree = M, priors_count = (1.0, 5.0] with effectiveness 90.00% and counterfactual cost = 2.0.
		Aggregate cost of the above recourses = 0.90
	Protected Subgroup 'Caucasian', 8.21% covered
		Make c_charge_degree = M with effectiveness 0.00% and counterfactual cost = 1.0.
		Make priors_count = (1.0, 5.0] with effectiveness 0.00% and counterfactual cost = 1.0.
		Make c_charge_degree = M, priors_count = (1.0, 5.0] with effectiveness 0.00% and counterfactual cost = 2.0.
		Aggregate cost of the above recourses = 0.00
	Bias against Caucasian. Unfairness score = 0.9.
If c_charge_degree = F, juv_fel_count = 0, juv_misd_count = 0, priors_count = (-0.1, 1

## Fairness of Recourse (using two-sample Kolmogorov-Smirnov test) -- Fair Effectiveness-Cost Trade-Off

In [15]:
# taken from the other notebooks
affected_pop_sizes = {"African-American": 483, "Caucasian": 262}

top_rules,unfairness = facts.select_rules_subset_KStest(
    rules_with_cumulative_correctness,
    affected_pop_sizes,
    top_count=top_count
)

print_recourse_report_KStest_cumulative(
    top_rules,
    population_sizes=affected_pop_sizes,
    unfairness = unfairness,
    show_then_costs=True
    # show_cumulative_plots=True
)

If juv_fel_count = 0, juv_other_count = 0, priors_count = (5.0, 10.0], sex = Male:
	Protected Subgroup 'African-American', 16.46% covered out of 483
		Make priors_count = (1.0, 5.0] with effectiveness 22.58% and counterfactual cost = 1.0.
		Make priors_count = (-0.1, 1.0] with effectiveness 97.85% and counterfactual cost = 1.0.
	Protected Subgroup 'Caucasian', 27.61% covered out of 262
		Make priors_count = (-0.1, 1.0] with effectiveness 100.00% and counterfactual cost = 1.0.
		Make priors_count = (1.0, 5.0] with effectiveness 100.00% and counterfactual cost = 1.0.
	 Unfairness based on the Kolmogorov-Smirnov test = 4.76.
If juv_fel_count = 0, priors_count = (5.0, 10.0], sex = Male:
	Protected Subgroup 'African-American', 18.58% covered out of 483
		Make priors_count = (1.0, 5.0] with effectiveness 20.95% and counterfactual cost = 1.0.
		Make priors_count = (-0.1, 1.0] with effectiveness 96.19% and counterfactual cost = 1.0.
	Protected Subgroup 'Caucasian', 29.10% covered out of 262
		

## Fairness of Effectiveness at Recourse Budget -- Equal Effectiveness within Budget (Micro)

In [16]:
top_rules, subgroup_costs = facts.select_rules_subset_cumulative(
    rules_with_cumulative_correctness,
    metric="max-upto-cost",
    sort_strategy="generic-sorting-ignore-exists-subgroup-empty",
    top_count=top_count,
    cost_threshold = cost_budget,
    filter_sequence = [
        # "remove-contained",
        # "remove-fair-rules",
        # "remove-above-thr-cost"
        #"remove-below-thr",
        #"keep-only-min-change",
        
    ],
    params=params
)

print_recourse_report_cumulative(
    top_rules,
    subgroup_costs=subgroup_costs,
    show_subgroup_costs=True,
    show_then_costs=True,
    correctness_metric = True
)

If age_cat = 25 - 45, juv_fel_count = 0, juv_other_count = 1, sex = Male:
	Protected Subgroup 'African-American', 5.13% covered
		Make juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 0.14.
		Make age_cat = Less than 25, juv_other_count = 0 with effectiveness 0.00% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 24.14% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 0.00
	Protected Subgroup 'Caucasian', 2.24% covered
		Make juv_other_count = 0 with effectiveness 100.00% and counterfactual cost = 0.14.
		Make age_cat = Less than 25, juv_other_count = 0 with effectiveness 100.00% and counterfactual cost = 10.14.
		Make age_cat = Greater than 45, juv_other_count = 0 with effectiveness 100.00% and counterfactual cost = 10.14.
		Aggregate cost of the above recourses = 1.00
	Bias against African-American. Unfairness score = 1.0.
If age_cat = 25 - 45, juv_fel_count = 0, juv_misd_count = 

## Get ranking of subgroups based on metrics

In [17]:
rules_with_both = {}
for ifc, all_thens in rules_with_cumulative_correctness.items():
    new_all_thens = {}
    for sg, (cov, thens_cum) in all_thens.items():
        thens_atom = rules_with_atomic_correctness[ifc][sg][1]
        thens_atom_dict = {then: atom_cor for then, atom_cor in thens_atom}
        new_all_thens[sg] = (cov, [(then, thens_atom_dict[then], cumcor) for then, cumcor, _cost in thens_cum])
    rules_with_both[ifc] = new_all_thens

In [18]:
from facts.fairness_metrics_aggr import make_table, auto_budget_calculation
budgets = auto_budget_calculation(rules_with_cumulative_correctness, cor_thres=0.5, percentiles=[0.3, 0.6, 0.9])
budgets

[1.0, 1.0, 10.0]

In [19]:
df = make_table(
    rules_with_both,
    sensitive_attribute_vals=["Caucasian", "African-American"],
    effectiveness_thresholds=[0.3, 0.7],
    cost_budgets=budgets,
    params=params
)

df.drop(['weighted-average', 'mean-cost-cinf'], axis=1, inplace=True)

In [20]:
df = df.loc[:, ~df.columns.duplicated()]
df



subgroup  \
                                               subgroup   
0                                   c_charge_degree = F   
1                       c_charge_degree = F, sex = Male   
2                c_charge_degree = F, juv_fel_count = 0   
3               c_charge_degree = F, juv_misd_count = 0   
4              c_charge_degree = F, juv_other_count = 0   
...                                                 ...   
1058  juv_fel_count = 0, juv_misd_count = 0, juv_oth...   
1059    juv_other_count = 2, priors_count = (-0.1, 1.0]   
1060  c_charge_degree = M, juv_fel_count = 0, juv_mi...   
1061  c_charge_degree = M, juv_misd_count = 0, sex =...   
1062  age_cat = Less than 25, priors_count = (1.0, 5...   

     (Equal Cost of Effectiveness(Macro), 0.3)                   \
                                     Caucasian African-American   
0                                          inf              inf   
1                                          inf              inf   
2                                          inf              inf   
3                                          inf              inf   
4                                          inf              inf   
...                                        ...              ...   
1058                                  0.285714         0.285714   
1059                                  0.285714         0.285714   
1060                                       inf              inf   
1061                                       inf              inf   
1062                                  1.000000         1.000000   

     (Equal Cost of Effectiveness(Macro), 0.7)                   \
                                     Caucasian African-American   
0                                          inf              inf   
1                                          inf              inf   
2                                          inf              inf   
3                                          inf              inf   
4                                          inf              inf   
...                                        ...              ...   
1058                                       inf              inf   
1059                                       inf         0.285714   
1060                                       inf              inf   
1061                                       inf              inf   
1062                                       1.0         1.000000   

     (Equal Choice for Recourse, 0.3)                   \
                            Caucasian African-American   
0                                   0                0   
1                                   0                0   
2                                   0                0   
3                                   0                0   
4                                   0                0   
...                               ...              ...   
1058                               -1               -1   
1059                               -1               -1   
1060                                0                0   
1061                                0                0   
1062                               -5               -5   

     (Equal Choice for Recourse, 0.7)                  Equal Effectiveness  \
                            Caucasian African-American           Caucasian   
0                                   0                0            0.196868   
1                                   0                0            0.205189   
2                                   0                0            0.218905   
3                                   0                0            0.209719   
4                                   0                0            0.227154   
...                               ...              ...                 ...   
1058                                0                0            0.333333   
1059                                0               -1            0.333333   


In [21]:
#table of differences between Male and Female for each metric
diff = get_diff_table(df, sensitive_attribute_vals=["Caucasian", "African-American"], with_abs = True)
diff = diff.set_index('subgroup')
ranked = diff.drop(columns=[('Fair Effectiveness-Cost Trade-Off','bias')]).mask(diff == 0).rank(ascending = False,axis=0,method='dense')
ranked = ranked.replace(np.nan,"Fair")

dropping on a non-lexsorted multi-index without a level parameter may impact performance.


In [22]:
rev_bias_metrics = ['Equal Effectiveness', 'Equal Effectiveness within Budget']
comb_df = get_comb_df(df, ranked, diff, rev_bias_metrics, sensitive_attribute_vals=["Caucasian", "African-American"])
analysis_df = get_analysis_df(comb_df, sensitive_attribute_vals=["Caucasian", "African-American"])

dropping on a non-lexsorted multi-index without a level parameter may impact performance.
The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


In [23]:
comb_df.head()

(Equal Cost of Effectiveness(Macro), 0.3)  \
                                                                              rank   
subgroup                                                                             
c_charge_degree = F                                                           Fair   
c_charge_degree = F, sex = Male                                               Fair   
c_charge_degree = F, juv_fel_count = 0                                        Fair   
c_charge_degree = F, juv_misd_count = 0                                       Fair   
c_charge_degree = F, juv_other_count = 0                                      Fair   

                                                             \
                                         score bias against   
subgroup                                                      
c_charge_degree = F                        0.0         Fair   
c_charge_degree = F, sex = Male            0.0         Fair   
c_charge_degree = F, juv_fel_count = 0     0.0         Fair   
c_charge_degree = F, juv_misd_count = 0    0.0         Fair   
c_charge_degree = F, juv_other_count = 0   0.0         Fair   

                                         (Equal Cost of Effectiveness(Macro), 0.7)  \
                                                                              rank   
subgroup                                                                             
c_charge_degree = F                                                           Fair   
c_charge_degree = F, sex = Male                                               Fair   
c_charge_degree = F, juv_fel_count = 0                                        Fair   
c_charge_degree = F, juv_misd_count = 0                                       Fair   
c_charge_degree = F, juv_other_count = 0                                      Fair   

                                                             \
                                         score bias against   
subgroup                                                      
c_charge_degree = F                        0.0         Fair   
c_charge_degree = F, sex = Male            0.0         Fair   
c_charge_degree = F, juv_fel_count = 0     0.0         Fair   
c_charge_degree = F, juv_misd_count = 0    0.0         Fair   
c_charge_degree = F, juv_other_count = 0   0.0         Fair   

                                         (Equal Choice for Recourse, 0.3)  \
                                                                     rank   
subgroup                                                                    
c_charge_degree = F                                                  Fair   
c_charge_degree = F, sex = Male                                      Fair   
c_charge_degree = F, juv_fel_count = 0                               Fair   
c_charge_degree = F, juv_misd_count = 0                              Fair   
c_charge_degree = F, juv_other_count = 0                             Fair   

                                                             \
                                         score bias against   
subgroup                                                      
c_charge_degree = F                        0.0         Fair   
c_charge_degree = F, sex = Male            0.0         Fair   
c_charge_degree = F, juv_fel_count = 0     0.0         Fair   
c_charge_degree = F, juv_misd_count = 0    0.0         Fair   
c_charge_degree = F, juv_other_count = 0   0.0         Fair   

                                         (Equal Choice for Recourse, 0.7)  \
                                                                     rank   
subgroup                                                                    
c_charge_degree = F                                                  Fair   
c_charge_degree = F, sex = Male                                      Fair   
c_charge_degree = F, juv_fel_count = 0                               Fair   
c_charge_degree = F, juv_misd_count = 0                              

In [24]:
analysis_df

,Rank = 1 Count,Caucasian bias against Count,African-American bias against Count
Equal(Conditional Mean Recourse),53,324,145
"(Equal Cost of Effectiveness(Macro), 0.3)",144,282,84
"(Equal Cost of Effectiveness(Micro), 0.3)",144,282,84
"(Equal Cost of Effectiveness(Macro), 0.7)",99,208,52
"(Equal Cost of Effectiveness(Micro), 0.7)",99,208,52
Equal Effectiveness,2,207,328
Fair Effectiveness-Cost Trade-Off,104,184,879
"(Equal Choice for Recourse, 0.7)",9,581,77
"(Equal Effectiveness within Budget, 1.0)",44,377,162
"(Equal Effectiveness within Budget, 10.0)",10,347,233
